In [11]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../src/')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from package.logger import Timed, rlog, setup
from package import storage
setup("WARNING")

In [13]:
import os

import folium
import geopandas as gpd
import pandas as pd

from package import strtime
from package.osm import osm
from package.mcr5.h3 import (
    get_h3_cells_for_nodes,
    get_h3_cells_for_bbox,
    plot_h3_cells_on_folium,
)
from package.mcr5.h3_osm_interaction import get_location_mappings_for_cells
from package.mcr5.mcr5 import MCR5
from package.osm.osm import add_nearest_osm_node_id
from package.mcr5.labels import read_labels_for_nodes
from package.minute_city import minute_city

from package.geometa import GeoMeta
from package.mcr.data import NetworkType, OSMData
from package.mcr.data import NetworkType

In [14]:
geo_meta_path = "../data/geometa.pkl"
city_id = "Koeln"
stops = "../data/cleaned/stops.csv"
structs = "../data/structs.pkl"
bicycle_location_path = "../data/bicycle_locations/2022-11-01_09_01_00.csv"
geo_meta = GeoMeta.load(geo_meta_path)
geo_data = OSMData(
	geo_meta,
	city_id,
	additional_network_types=[NetworkType.DRIVING],
)
nodes = geo_data.osm_nodes
edges = geo_data.osm_edges
driving_nodes, _, _ = geo_data.additional_networks[NetworkType.DRIVING]

In [15]:
h3_cells = get_h3_cells_for_nodes(nodes[["lat", "lon"]].to_dict("records"), 9)
driving_h3_cells = get_h3_cells_for_nodes(driving_nodes[["lat", "lon"]].to_dict("records"), 9)
h3_cells = h3_cells.intersection(driving_h3_cells)
m = folium.Map(location=[50.9333, 6.95], zoom_start=12)
plot_h3_cells_on_folium(h3_cells, m)
m

In [16]:
location_mappings, invalid_h3_cells = get_location_mappings_for_cells(list(h3_cells), nodes, 20)
invalid_h3_cells

[]

In [17]:
m = folium.Map(location=[50.9375, 6.9603], zoom_start=12)
plot_h3_cells_on_folium(map(lambda lm: lm.h3_cell, location_mappings), m)
nodes_by_id = nodes.set_index("id")
for location_mapping in location_mappings:
	node = nodes_by_id.loc[location_mapping.osm_node_id]
	folium.CircleMarker(
		location=(node.lat, node.lon),
		icon=folium.Icon(color="green"),
		radius=1,
		color="red",
	).add_to(m)
m

In [20]:

car_location_mappings, invalid_h3_cells = get_location_mappings_for_cells(
    list(h3_cells), driving_nodes, max_tries=40
)
invalid_h3_cells

[]

In [9]:
import h3

In [10]:
m = folium.Map(location=[50.9375, 6.9603], zoom_start=12)
plot_h3_cells_on_folium(map(lambda lm: lm.h3_cell, car_location_mappings), m)
nodes_by_id = driving_nodes.set_index("id")


for location_mapping in car_location_mappings:
	node = nodes_by_id.loc[location_mapping.osm_node_id]
	folium.CircleMarker(
		location=(node.lat, node.lon),
		radius=1,
		color="red",
	).add_to(m)
m